In [17]:
from collections import namedtuple
import os
os.chdir("../")

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) 
class PrepareCallbackConfig:      #this will also work as a named tuple but is a better way to implement the same concept
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callback
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)   #since it has a filename in the end of filepath
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_log_dir)])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_log_dir = Path(config.tensorboard_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath))

        return prepare_callback_config

In [32]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):   #tensorboard callbacks
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")  #good practice to create folders with timestamp names as they are unique
        tb_running_log_dir = os.path.join(self.config.tensorboard_log_dir,(f"tb_logs_at_{timestamp}"))
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):   #checkpoint callbacks    
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath)


    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,   #we dont need to call a property as method i.e. with ()
            self._create_ckpt_callbacks
        ]

In [33]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

In [23]:
os.chdir("Deep Learning Project")